In [ ]:
class Nodo:
  def __init__(self, valor):
    self.valor = valor
    self.siguiente = None
    self.anterior = None

In [ ]:
#Clase representa una cita individual
class Cita:
    contador_id = 1  # id incremental

    def __init__(self, nombrePaciente, doctor, hora, estado="Programada"):
        self.id = Cita.contador_id
        Cita.contador_id += 1
        self.nombrePaciente = nombrePaciente
        self.doctor = doctor
        self.hora = hora
        self.estado = estado
        self.siguiente = None
        self.anterior = None

    def __str__(self):
        return f"ID: {self.id} | Paciente: {self.nombrePaciente} | Doctor: {self.doctor} | Hora: {self.hora} | Estado: {self.estado}"

Esta clase gestionará la colección de todas las citas registradas.
Implementará métodos para:
Agregar una nueva cita.
Buscar una cita por su ID.
Modificar los datos de una cita.
Cambiar el estado de una cita (por ejemplo, al cancelar).
Listar todas las citas.



In [ ]:
class ListaEnlazadaCitas:
    def __init__(self):
        self.cabeza = None
        self.cola = None
        self.size = 0

    def agregarCita(self, cita):
        nuevo_nodo = Nodo(cita)
        if not self.cabeza:
            self.cabeza = self.cola = nuevo_nodo
        else:
            self.cola.siguiente = nuevo_nodo
            nuevo_nodo.anterior = self.cola
            self.cola = nuevo_nodo
        self.size += 1

    def buscarCitaId(self, id):
        actual = self.cabeza
        while actual:
            if actual.valor.id == id:
                return actual.valor
            actual = actual.siguiente
        return None

    def modificarCita(self, cita_modificada):
        actual = self.cabeza
        while actual:
            if actual.valor.id == cita_modificada.id:
                actual.valor = cita_modificada
                return True
            actual = actual.siguiente
        return False

    def cambiarEstadoCita(self, id, nuevoEstado):
        cita = self.buscarCitaId(id)
        if cita:
            cita.estado = nuevoEstado
            return True
        return False

    def listarCitas(self):
        citas = []
        actual = self.cabeza
        while actual:
            citas.append(actual.valor)
            actual = actual.siguiente
        return citas

    def eliminarCitaPorId(self, id):
        actual = self.cabeza
        while actual:
            if actual.valor.id == id:
                if actual.anterior:
                    actual.anterior.siguiente = actual.siguiente
                else:
                    self.cabeza = actual.siguiente
                if actual.siguiente:
                    actual.siguiente.anterior = actual.anterior
                else:
                    self.cola = actual.anterior
                self.size -= 1
                return True
            actual = actual.siguiente
        return False

    def __iter__(self):
        actual = self.cabeza
        while actual:
            yield actual.valor
            actual = actual.siguiente

Esta clase gestionará las citas que están pendientes de ser procesadas, respetando el orden de llegada (FIFO - Primero en Entrar, Primero en Salir).
Implementará métodos para:
Encolar una nueva cita (agregarla al final de la cola).
Desencolar una cita (obtener y eliminar la cita del frente de la cola para procesarla).
Verificar si la cola está vacía.
Listar las citas en la cola.


In [ ]:
class ColaCitasPendientes:
    def __init__(self):
        self.items = ListaEnlazadaCitas()

    def encolarCita(self, cita):
        self.items.agregarCita(cita)

    def desencolarCita(self):
        if not self.estaVacia():
            primera_cita = self.items.cabeza.valor
            self.items.eliminarCitaPorId(primera_cita.id)
            return primera_cita
        else:
            raise IndexError("La cola está vacía")

    def estaVacia(self):
        return self.items.size == 0

    def listarCitas(self):
        print("Citas pendientes:")
        for cita in self.items:
            print(f"ID: {cita.id} | Paciente: {cita.nombrePaciente} | Doctor: {cita.doctor} | Hora: {cita.hora} | Estado: {cita.estado}")


In [ ]:
class PilaHistorialOperaciones:
    def __init__(self):
        self.items = []

    def apilarOperacion(self, operacion):
        self.items.append(operacion)

    def desapilarUltimaOperacion(self):
        if not self.estaVacia():
            return self.items.pop()
        else:
            raise IndexError("La pila está vacía")

    def estaVacia(self):
        return len(self.items) == 0

    def listarOperaciones(self):
        print("Historial de operaciones:")
        for operacion in reversed(self.items):
            print(operacion)

In [ ]:
class Hospital:
    def __init__(self):
        self.lista_citas = ListaEnlazadaCitas()
        self.cola_pendientes = ColaCitasPendientes()
        self.historial = PilaHistorialOperaciones()

    def registrarCita(self, nombrePaciente, doctor, hora):
        nueva_cita = Cita(nombrePaciente, doctor, hora)
        self.lista_citas.agregarCita(nueva_cita)
        self.cola_pendientes.encolarCita(nueva_cita)
        self.historial.apilarOperacion({"tipo": "creación", "cita": nueva_cita})
        return nueva_cita

    def procesarCita(self):
        if not self.cola_pendientes.estaVacia():
            cita_procesada = self.cola_pendientes.desencolarCita()
            self.lista_citas.cambiarEstadoCita(cita_procesada.id, "Completada")
            self.historial.apilarOperacion({"tipo": "procesamiento", "cita": cita_procesada})
            return cita_procesada
        else:
            raise IndexError("No hay citas pendientes para procesar.")

    def modificarCita(self, id, nuevoNombre=None, nuevoDoctor=None, nuevaHora=None):
        cita = self.lista_citas.buscarCitaId(id)
        if cita:
            # Guarda los datos originales para poder revertir
            datos_originales = {
                "id": cita.id,
                "nombrePaciente": cita.nombrePaciente,
                "doctor": cita.doctor,
                "hora": cita.hora,
            }
            if nuevoNombre:
                cita.nombrePaciente = nuevoNombre
            if nuevoDoctor:
                cita.doctor = nuevoDoctor
            if nuevaHora:
                cita.hora = nuevaHora
            self.lista_citas.modificarCita(cita)
            self.historial.apilarOperacion({"tipo": "modificación", "cita": cita, "original": datos_originales})
            return cita
        else:
            raise ValueError("No se encontró una cita con ese ID.")

    def cancelarCita(self, id):
        cita = self.lista_citas.buscarCitaId(id)
        if cita:
            self.lista_citas.cambiarEstadoCita(id, "Cancelada")
            self.historial.apilarOperacion({"tipo": "cancelación", "cita": cita})
            return True
        else:
            raise ValueError("No se encontró una cita con ese ID.")

    def deshacerUltimaOperacion(self):
        if self.historial.estaVacia():
            print("No hay operaciones para deshacer.")
            return

        ultima_operacion = self.historial.desapilarUltimaOperacion()
        tipo = ultima_operacion["tipo"]

        if tipo == "creación":
            # Eliminar la cita creada
            cita = ultima_operacion["cita"]
            self.lista_citas.eliminarCitaPorId(cita.id)
            self.cola_pendientes.items.eliminarCitaPorId(cita.id)
            print(f"Se deshizo la creación de la cita: {cita}")

        elif tipo == "procesamiento":
            # Revertir el procesamiento de la cita
            cita = ultima_operacion["cita"]
            self.lista_citas.cambiarEstadoCita(cita.id, "Programada")
            self.cola_pendientes.encolarCita(cita)
            print(f"Se deshizo el procesamiento de la cita: {cita}")

        elif tipo == "modificación":
            # Revertir la modificación de la cita
            cita = ultima_operacion["cita"]
            datos_originales = ultima_operacion["original"]
            cita.nombrePaciente = datos_originales["nombrePaciente"]
            cita.doctor = datos_originales["doctor"]
            cita.hora = datos_originales["hora"]
            self.lista_citas.modificarCita(cita)
            print(f"Se deshizo la modificación de la cita: {cita}")

        elif tipo == "cancelación":
            # Revertir la cancelación de la cita
            cita = ultima_operacion["cita"]
            self.lista_citas.cambiarEstadoCita(cita.id, "Programada")
            self.cola_pendientes.encolarCita(cita)
            print(f"Se deshizo la cancelación de la cita: {cita}")

In [ ]:
class Vista:
    def __init__(self):
        self.hospital = Hospital()

    def mostrarMenu(self):
        print("***Bienvenido al menú principal del Hospital UAM***")
        print("Opciones disponibles para seleccionar:")
        print("1. Registrar cita")
        print("2. Procesar cita")
        print("3. Modificar cita")
        print("4. Cancelar cita")
        print("5. Ver historial de operaciones")
        print("6. Deshacer última operación")
        print("7. Salir del sistema")

        opc = input("Ingrese una opcion, administrador: ")

        match opc:
            case "1":
                nombrePaciente = input("Ingrese el nombre del paciente: ")
                doctor = input("Ingrese el nombre del doctor: ")
                hora = input("Ingrese la hora de la cita (formato HH:MM AM/PM): ")
                cita = self.hospital.registrarCita(nombrePaciente, doctor, hora)
                print(f"Cita registrada exitosamente: {cita}")

            case "2":
                try:
                    cita = self.hospital.procesarCita()
                    print(f"Cita procesada: {cita}")
                except IndexError as e:
                    print(e)

            case "3":
                try:
                    idCita = int(input("Ingrese el ID de la cita a modificar: "))
                    nuevoNombre = input("Nuevo nombre del paciente (Enter para no cambiar): ")
                    nuevoDoctor = input("Nuevo nombre del doctor (Enter para no cambiar): ")
                    nuevaHora = input("Nueva hora de la cita (Enter para no cambiar): ")
                    cita = self.hospital.modificarCita(idCita, nuevoNombre, nuevoDoctor, nuevaHora)
                    print(f"Cita modificada exitosamente: {cita}")
                except ValueError as e:
                    print(e)

            case "4":
                try:
                    idCita = int(input("Ingrese el ID de la cita a cancelar: "))
                    self.hospital.cancelarCita(idCita)
                    print("Cita cancelada exitosamente.")
                except ValueError as e:
                    print(e)

            case "5":
                self.hospital.listarHistorial()

            case "6":
                self.hospital.deshacerUltimaOperacion()

            case "7":
                print("Saliendo del sistema. ¡Gracias por usar el sistema del Hospital UAM!")
                return

            case _:
                print("Opción no válida. Por favor, seleccione una opción del menú.")

In [ ]:
if __name__ == "__main__":
    lista_citas = ListaEnlazadaCitas()
    cola_pendientes = ColaCitasPendientes()
    historial = PilaHistorialOperaciones()

    cita1 = Cita("Ana juana", "Dr. Ramírez", "08:00 AM")
    cita2 = Cita("Luis Pepe", "Dra. Salas", "09:30 AM")
    cita3 = Cita("Diego Norrea", "Dr. Pérez", "10:15 AM")

    lista_citas.agregarCita(cita1)
    lista_citas.agregarCita(cita2)
    lista_citas.agregarCita(cita3)

    cola_pendientes.encolarCita(cita1)
    cola_pendientes.encolarCita(cita2)
    cola_pendientes.encolarCita(cita3)

    historial.apilarOperacion({"tipo": "creación", "cita": cita1})

    print("- TODAS LAS CITAS")
    for cita in lista_citas:
        print(cita)

    print("- CITAS PENDIENTES")
    cola_pendientes.listarCitas()

    print("- HISTORIAL")
    historial.listarOperaciones()

    procesada = cola_pendientes.desencolarCita()
    lista_citas.cambiarEstadoCita(procesada.id, "Completada")
    print(f"- Cita procesada:{procesada}")

    print("- CITAS PENDIENTES (Despues)")
    cola_pendientes.listarCitas()


- TODAS LAS CITAS
ID: 10 | Paciente: Ana juana | Doctor: Dr. Ramírez | Hora: 08:00 AM | Estado: Programada
ID: 11 | Paciente: Luis Pepe | Doctor: Dra. Salas | Hora: 09:30 AM | Estado: Programada
ID: 12 | Paciente: Diego Norrea | Doctor: Dr. Pérez | Hora: 10:15 AM | Estado: Programada
- CITAS PENDIENTES
Citas pendientes:
ID: 10 | Paciente: Ana juana | Doctor: Dr. Ramírez | Hora: 08:00 AM | Estado: Programada
ID: 11 | Paciente: Luis Pepe | Doctor: Dra. Salas | Hora: 09:30 AM | Estado: Programada
ID: 12 | Paciente: Diego Norrea | Doctor: Dr. Pérez | Hora: 10:15 AM | Estado: Programada
- HISTORIAL
Historial de operaciones:
{'tipo': 'creación', 'cita': <__main__.Cita object at 0x7e8bb9718b10>}
- Cita procesada:ID: 10 | Paciente: Ana juana | Doctor: Dr. Ramírez | Hora: 08:00 AM | Estado: Completada
- CITAS PENDIENTES (Despues)
Citas pendientes:
ID: 11 | Paciente: Luis Pepe | Doctor: Dra. Salas | Hora: 09:30 AM | Estado: Programada
ID: 12 | Paciente: Diego Norrea | Doctor: Dr. Pérez | Hora: 1